In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Load dataset


In [42]:
import urllib.request
import os
import zipfile

url = "https://drive.google.com/file/d/1Ok23WNlNjIpy0PdoD2JiM_jqE6gmGrVx/view?usp=sharing"
target = "/content/drive/MyDrive/DSLab/Session 3/data/data.zip"

if not os.path.exists(target):

    urllib.request.urlretrieve(url, target)



In [48]:
!unzip  "/content/drive/MyDrive/DSLab/Session 3/data/data.zip" -d "/content/drive/MyDrive/DSLab/Session 3/data/"

Archive:  /content/drive/MyDrive/DSLab/Session 3/data/data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/DSLab/Session 3/data/data.zip or
        /content/drive/MyDrive/DSLab/Session 3/data/data.zip.zip, and cannot find /content/drive/MyDrive/DSLab/Session 3/data/data.zip.ZIP, period.


In [ ]:
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import tensorflow as tf
import random




Instructions for updating:
non-resource variables are not supported in the long term


1. Tạo class MLP để xây dựng kiến trúc multi-layer perception gồm 1 layer input, 1 hidden layer, 1 layer output và định nghĩa 1 số hàm: build_graph, trainer

In [ ]:
class MLP:
  def __init__(self, vocab_size, hidden_size_1):
    self._vocab_size = vocab_size
    self._hidden_size_1 = hidden_size_1

  def build_graph(self):
    self._X = tf.placeholder(tf.float32, shape= [None, self._vocab_size])
    self._real_Y = tf.placeholder(tf.int32, shape=[None, ])
    NUM_CLASSES = 20

    weights_1 = tf.get_variable(
        name= 'weights_input_hidden_1',
        shape= (self._vocab_size, self._hidden_size_1),
        initializer= tf.random_normal_initializer(seed=2021),
    )
    biases_1 = tf.get_variable(
        name= 'biases_input_hidden_1',
        shape= (self._hidden_size_1),
        initializer= tf.random_normal_initializer(seed= 2021)
    )

    weights_2 = tf.get_variable(
        name= 'weights_hidden_1_2',
        shape= (self._hidden_size_1, NUM_CLASSES),
        initializer= tf.random_normal_initializer(seed=2021)
    ) 
    biases_2 = tf.get_variable(
        name= 'biases_hidden_1_2',
        shape= (NUM_CLASSES),
        initializer= tf.random_normal_initializer(seed= 2021)
    )

    # weights_2 = tf.get_variable(
    #     name= 'weights_hidden_1_2',
    #     shape= (self._hidden_size_1, self._hidden_size_2),
    #     initializer= tf.random_normal_initializer(seed=2021)
    # ) 
    # biases_2 = tf.get_variable(
    #     name= 'biases_hidden_1_2',
    #     shape= (self._hidden_size_2),
    #     initializer= tf.random_normal_initializer(seed= 2021)
    # )
    # weights_3 = tf.get_variable(
    #     name= 'weights_hidden_2_3',
    #     shape= (self._hidden_size_2, self._hidden_size_3),
    #     initializer= tf.random_normal_initializer(seed=2021)
    # ) 
    # biases_3 = tf.get_variable(
    #     name= 'biases_hidden_2_3',
    #     shape= (self._hidden_size_3),
    #     initializer= tf.random_normal_initializer(seed= 2021)
    # )

    # weights_4 = tf.get_variable(
    #     name= 'weights_hidden_3_output',
    #     shape= (self._hidden_size_3, NUM_CLASSES),
    #     initializer= tf.random_normal_initializer(seed=2021)
    # )
    # biases_4 = tf.get_variable(
    #     name= 'biases_hidden_3_output',
    #     shape= (NUM_CLASSES),
    #     initializer= tf.random_normal_initializer(seed= 2021)
    # )
    hidden_1 = tf.matmul(self._X, weights_1) + biases_1
    hidden_1 = tf.nn.relu(hidden_1)

    # hidden_2 = tf.matmul(hidden_1, weights_2) + biases_2
    # hidden_2 = tf.nn.relu(hidden_2)

    # hidden_3 = tf.matmul(hidden_2, weights_3) + biases_3
    # hidden_3 = tf.nn.relu(hidden_3)


    logits = tf.matmul(hidden_1, weights_2) + biases_2

    labels_one_hot = tf.one_hot(
        indices= self._real_Y,
        depth= NUM_CLASSES,
        dtype= tf.float32
    )
    loss = tf.nn.softmax_cross_entropy_with_logits(labels= labels_one_hot,
                                                   logits= logits)
    loss = tf.reduce_mean(loss)

    probs = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(probs, axis= 1)
    predicted_labels = tf.squeeze(predicted_labels)

    return predicted_labels, loss
  def trainer(self, loss, learning_rate):
    train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_opt
  

2. Tạo lớp DataReader để đọc dữ liệu

In [ ]:
class DataReader:
  def __init__(self, data_path, batch_size, vocab_size):
    self._batch_size = batch_size
    
    with open(data_path) as f:
      d_lines = f.read().splitlines()
    self._data = []
    self._labels = []
    for data_id, line in enumerate(d_lines):
        vector = [0.0 for _ in range(vocab_size)]
        features = line.split('<fff>')
        label, doc_id = int(features[0]), int(features[1])
        
        tokens = features[2].split()
        for token in tokens:
          index, value = int(token.split(':')[0]),\
                             float(token.split(':')[1])
          vector[index] = value

        self._data.append(vector)
        self._labels.append(label)
    self._data = np.array(self._data)
    self._labels = np.array(self._labels)

    self._num_epoch = 0
    self._batch_id = 0

  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1

    if end + self._batch_size > len(self._data):
      end = len(self._data)
      self._num_epoch += 1
      self._batch_id = 0
      indices = list(range(len(self._data)))
      random.seed(2018)
      random.shuffle(indices)
      self._data, self._labels = self._data[indices], self._labels[indices]
    return self._data[start:end], self._labels[start:end]


3. Định nghĩa 1 số hàm: 
  - load_dataset(): đọc dữ liệu đưa và quá trình train và test
  - save_parameter(): lưu trọng số sau quá trình train
  - restore_parameter(): nạp trọng số trong quá trình test

In [ ]:
def load_dataset():
  train_data_reader = DataReader(
      data_path= '/content/drive/MyDrive/DSLab/Session 3/data/train_tf_idf.txt',
      batch_size= 50,
      vocab_size= vocab_size
  )
  test_data_reader = DataReader(
      data_path= '/content/drive/MyDrive/DSLab/Session 3/data/test_tf_idf.txt',
      batch_size= 50,
      vocab_size= vocab_size
  )
  return train_data_reader, test_data_reader
def save_parameter(name, value, epoch):
  filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
  if len(value.shape) == 1:
    string_form = ','.join([str(number) for number in value])
  else:
    string_form = '\n'.join([','.join([str(number) for number in value[row]])
                      for row in range(value.shape[0])])
  with open('/content/drive/MyDrive/DSLab/Session 3/save_parameters/' + filename, 'w') as f:
    f.write(string_form)
  
def restore_parameter(name, epoch):
  filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
  with open('/content/drive/MyDrive/DSLab/Session 3/save_parameters/' + filename) as f:
    lines = f.read().splitlines()
  
  if len(lines) == 1:
    value = [float(number) for number in lines[0].split(',')]
  else:
    value = [[float(number) for number in lines[row].split(',')]
             for row in range(len(lines))]
  return value

4. Mở 1 phiên chạy thực hiện training

In [ ]:
with open('/content/drive/MyDrive/DSLab/Session 3/data/words_idfs.txt') as f:
  vocab_size = len(f.read().splitlines())

mlp = MLP(
    vocab_size= vocab_size,
    hidden_size_1= 50
)

tf.reset_default_graph()
predicted_labels, loss = mlp.build_graph()
train_opt = mlp.trainer(loss= loss, learning_rate= 0.1)
train_data_reader, test_data_reader = load_dataset()


with tf.Session() as sess:
  
  step, MAX_STEP = 0, 100000
  print(len(train_data_reader._data))

  sess.run(tf.global_variables_initializer())
  while step < MAX_STEP:
    train_data, train_labels = train_data_reader.next_batch()
    plabels_eval, loss_eval, _ = sess.run(
        [predicted_labels, loss, train_opt],
        feed_dict= {
            mlp._X: train_data,
            mlp._real_Y: train_labels
        }
    )
    step += 1
    print('Step: {}, loss: {}'.format(step, loss_eval))
  
  trainable_varibles = tf.trainable_variables()
  for variable in trainable_varibles:
    save_parameter(
        name= variable.name,
        value= variable.eval(),
        epoch= train_data_reader._num_epoch
    )
  






Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Step: 95001, loss: 0.12950149178504944
Step: 95002, loss: 0.15600933134555817
Step: 95003, loss: 0.18207824230194092
Step: 95004, loss: 0.1292862743139267
Step: 95005, loss: 0.09533189982175827
Step: 95006, loss: 0.09228413552045822
Step: 95007, loss: 0.11983690410852432
Step: 95008, loss: 0.1723715215921402
Step: 95009, loss: 0.2551070749759674
Step: 95010, loss: 0.2497219443321228
Step: 95011, loss: 0.04062339663505554
Step: 95012, loss: 0.0
Step: 95013, loss: 0.08981812745332718
Step: 95014, loss: 0.043386753648519516
Step: 95015, loss: 0.0
Step: 95016, loss: 0.0
Step: 95017, loss: 0.0
Step: 95018, loss: 0.0
Step: 95019, loss: 0.19336219131946564
Step: 95020, loss: 0.09686721861362457
Step: 95021, loss: 0.17219330370426178
Step: 95022, loss: 0.038173526525497437
Step: 95023, loss: 0.031384486705064774
Step: 95024, loss: 0.09637635946273804
Step: 95025, loss: 0.0
Step: 95026, loss: 0.2938445210456848
Step: 95027, loss: 0.212005

In [ ]:
# test_data_reader = DataReader(
#     data_path= '/content/drive/MyDrive/DSLab/Session 3/data/test_tf_idf.txt',
#     batch_size= 50,
#     vocab_size= vocab_size
# )
with tf.Session() as sess:
  epoch = 442
  trainable_varibles = tf.trainable_variables()
  for variable in trainable_varibles:
    saved_value = restore_parameter(variable.name, epoch)
    assign_op = variable.assign(saved_value)
    sess.run(assign_op)
  
  num_true_preds = 0
  while True:
    test_data, test_labels = test_data_reader.next_batch()
    test_plables_eval = sess.run(
        predicted_labels,
        feed_dict= {
            mlp._X: test_data,
            mlp._real_Y: test_labels
        }
    )
    matches = np.equal(test_plables_eval, test_labels)
    num_true_preds += np.sum(matches.astype(float))

    if test_data_reader._batch_id == 0:
      break
  
  print('Epoch: ', epoch)
  print('Accuracy on test data: ', num_true_preds / len(test_data_reader._data))


Epoch:  442
Accuracy on test data:  0.7193308550185874


Epoch:  442
Accuracy on test data:  0.7193308550185874